# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/unsloth/Qwen3-32B-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 30, 2, 52, 32, 372570, tzinfo=TzInfo(-05:00)),
   'digest': '77e7d26c05df9acac668c43ac2f3f6f2545110ae0ad88ec0bc48c93ea5e6675a',
   'size': 19762152260,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'qwen3',
    'families': ['qwen3'],
    'parameter_size': '32.8B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=Tz

In [2]:
import ollama

model = "gpt-oss:latest"

# ollama.pull(model) # uncomment this if you need to download the model

# generating a new story

In [3]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

completions = []
story_variables = []

In [4]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)

completions.append(response.model_dump())
premise_opening = response.message.content

Markdown(response.message.content)

Premise:  
In a floating metropolis that hovers over a ruined Earth, an unassuming elevator engineer discovers that the city’s failing core is linked to a forgotten royal bloodline. When a sudden collapse threatens the entire skyward island, he must dig into his buried past to find the ancient mechanism that can stabilize the core. But the only key to that mechanism is hidden in the memories he has deliberately erased—memories that could undo the very society he protects.

Opening paragraph:  
The elevator doors hiss open with the same stale, copper scent that clung to my childhood kitchen. Below me, the sky‑city’s endless sprawl glows like a constellation of lanterns, each street a filament of light. I press the panel button and the shaft groans, a low, metallic sigh that feels like the city exhaling. I’ve spent years fixing these veins, patching the cracks that let the wind in. Yet tonight the shaft shuddered, sending a surge of electricity across my fingertips, and the glass in the window trembled, showing me, for the first time, a ghostly outline of a door that doesn’t belong to any floor. The world outside the window quivered—was it the building, or something else entirely?

In [5]:
import re

text = premise_opening  # your input string

pattern = re.compile(r"""
^\s*
(?:[\*_]{1,3}\s*)?                                 # optional opening markdown (** or *)
(?P<label>premise|opening\s+paragraph|original\s+paragraph)
[\s\*_]*[:：\-—–]?[\s\*_]*                          # optional punctuation/spacing around label
(?P<body>.+?)                                      # capture the body non-greedily
(?=
    ^\s*(?:[\*_]{1,3}\s*)?
        (?:premise|opening\s+paragraph|original\s+paragraph)[\s\*_]*[:：\-—–]?  # next labeled section
  | \n{2,}(?=\S)                                   # or a blank-line block then non-space
  | \Z                                             # or end of string
)
""", flags=re.IGNORECASE | re.MULTILINE | re.DOTALL | re.VERBOSE)

label_norm = {
    "premise": "Premise",
    "opening paragraph": "Opening paragraph",
    "original paragraph": "Original paragraph",
}

sections = {}
for m in pattern.finditer(text):
    raw_label = re.sub(r"\s+", " ", m.group("label").lower()).strip()
    label = label_norm.get(raw_label, raw_label.title())
    body = m.group("body").strip()
    sections[label] = f"{label}: {body}"

premise   = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

story_variables.append({"premise": premise})
story_variables.append({"paragraph": paragraph})

print(premise)
print(paragraph)


Premise: In a floating metropolis that hovers over a ruined Earth, an unassuming elevator engineer discovers that the city’s failing core is linked to a forgotten royal bloodline. When a sudden collapse threatens the entire skyward island, he must dig into his buried past to find the ancient mechanism that can stabilize the core. But the only key to that mechanism is hidden in the memories he has deliberately erased—memories that could undo the very society he protects.
Opening paragraph: The elevator doors hiss open with the same stale, copper scent that clung to my childhood kitchen. Below me, the sky‑city’s endless sprawl glows like a constellation of lanterns, each street a filament of light. I press the panel button and the shaft groans, a low, metallic sigh that feels like the city exhaling. I’ve spent years fixing these veins, patching the cracks that let the wind in. Yet tonight the shaft shuddered, sending a surge of electricity across my fingertips, and the glass in the windo

In [6]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [7]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)

completions.append(response.model_dump())
continued_paragraph = response.message.content

story_variables.append({"continued_paragraph":continued_paragraph})

Markdown(response.message.content)

I swallowed the copper‑tinged air and pressed the panel again. The elevator shaft, a skeletal spine of humming wires, seemed to pulse in time with my heartbeat. When the ghost door flickered—a translucent outline that matched no floor plan—my fingers tightened around the control wheel. A hiss of metal sounded like a sigh. I glanced at the digital readout: the core power levels were dipping at a rate that would collapse the sky‑city in under three hours.

The city’s sirens—soft, melodic alarms that usually announced maintenance—now rang out in a discordant chord. A low voice in the PA, filtered through the elevator’s speakers, chirped in the city’s synthetic tongue: *“Attention, citizens. Core stabilization protocols are in full activation. Please refrain from using the elevators. Evacuation to the lower decks is recommended.”* I heard a faint hiss of dissent in the same voice—like the system itself was pleading.

I pushed the elevator door open. Inside the shaft, I saw the ghost door again, its outline shifting in the low light, like a second set of stairs that no one had built. I stepped into the shaft, the cold metal of the rails biting my skin. The ghost door was at the very bottom, a small, matte black panel, almost flush with the wall. A thin seam ran along its edge, pulsing faintly with a blue light that I had never seen before.

It was a keyhole of sorts, but it didn’t look like any lock I’d worked on. Instead of a tumblers, there was a shallow groove, a single slot that seemed to be a conduit for something else. I lifted my gloved hand and placed my fingers along the groove. The elevator’s wiring hummed in response. The groove, when pressed, released a low vibration that traveled along the shaft, reverberating through the metal like a prayer. My fingertips tingled.

I pressed harder. The groove answered, and the panel slid open on a silent, mechanical creak. A narrow, dim corridor opened up, hidden beneath the elevator’s maintenance hatch. Dust motes danced in the weak light, and the air smelled of old metal and ozone. My torchlight cut a pale circle in the corridor, revealing a wall covered in etched symbols. A stone tablet lay half buried in the floor, covered with a thin layer of grime that had been untouched for centuries.

I brushed the dirt away and read the inscription. The script was unfamiliar, but the shape of the letters—curved, interlocking—seemed oddly familiar. I traced the glyphs with my finger. It read, in an ancient dialect of the city’s language, *“When the sky falls, the blood of the sky will rise.”* Beneath that, in a smaller, more modern script, was an address in the city’s core, the coordinates of the central power unit.

My mind flooded with images that had been buried beneath layers of state‑controlled memory. A small, flickering image: my grandmother, her hair wild, her eyes bright with defiance. She whispered to me, in a language that was not recorded by the city’s official archives, that we were descendants of a line that once engineered the core. I’d never heard that before; I’d only known the old myths about the city’s founding, stories of the sky and the sea that never existed. But now I could see it: my own name, written in the ancient script, on the stone tablet. The memory had been erased, a cut in my mind where the truth lay.

A sudden thud echoed from deeper in the shaft. I turned and saw a small, dark shape move, then vanish. I realized the elevator shaft wasn’t just a maintenance lane; it was a conduit to something older, something that had survived the collapse of the city’s foundations.

I had to find the memory recovery center. The city’s records said that only a handful of technicians were trained to retrieve lost memories, and even they could only do so with great risk. I’d heard rumors that the memory retrieval process could trigger a cascade of repressed memories, sometimes causing people to lose their sense of self entirely. But I didn’t have a choice. The city was about to collapse.

I made my way down the narrow corridor, each step bringing me closer to the core. The torchlight flickered, casting elongated shadows that danced against the stone walls. At the end of the passage, I found a door that bore the symbol of the royal crest, the same emblem that had been etched into the stone tablet. The door was locked, but the lock was a simple slot, not a tumblers, but a keyhole that fit my gloved finger perfectly. I pressed my hand against it and felt a sudden jolt of energy. The door swung open on its own, revealing a small, dimly lit chamber.

Inside the chamber was a set of reclining chairs, a low table, and a small console that hummed softly. A technician, a woman with silver hair and eyes that looked like storm clouds, sat in a chair, her hands resting on the console. She looked up as I entered, her face reflecting a mixture of caution and pity.

“Provisional access granted,” she said. “But you must understand the cost. Memories are not simply recorded; they are experiences. Retrieving them is like plucking a string from a violin; the sound will be there, but the string will be broken.”

I nodded. “I understand.”

She pressed a button. A light flickered on the console, and a soft, green glow enveloped the room. I felt my body tense. I closed my eyes and let the technician guide me. She whispered a chant in the ancient dialect, and a low hum resonated through my bones. Images burst into my mind—my grandmother’s voice, a hidden room in my family home, a key hidden in the roots of a tree. The memory flooded, and with it, the knowledge of the mechanism hidden in the core. The key was not a physical object; it was a code—a pattern of pressure that only a descendant of the royal line could understand. The pattern lay in my heart, in the way I’d instinctively turned the elevator panel in a rhythm that matched the core’s pulse.

The technician’s voice cut through the rush of images. “You must find the core. The mechanism is hidden in the main power unit. The key is your bloodline’s rhythm.”

I opened my eyes. The green glow dimmed, and the chamber’s walls seemed to pulse with a faint, metallic thrum. I could feel the weight of the city’s fate pressing on me. The core would fail in a matter of hours unless I could locate the mechanism and activate it. The key was my lineage; the mechanism was hidden in the heart of the sky‑city. I understood that the memory I had erased was a safeguard, a truth the city had hidden to keep its citizens from the knowledge of their own power. The same truth now threatened to collapse the city itself.

I turned to leave, but the ghost door, the black panel at the bottom of the shaft, seemed to call to me. Its outline flickered brighter, as if it were a pulse of the core itself. I stepped back into the shaft, the elevator doors closing softly behind me, sealing me in a world of humming wires and ancient secrets.

I walked to the ghost door. I pressed the edge of the groove again, feeling the same vibration that had opened the hidden corridor. The panel slid with a whisper, revealing a small, cylindrical mechanism embedded in the wall—an ancient cog, its surface etched with the same symbols as the stone tablet. I lifted it, and the city’s hum seemed to rise and fall in a steady rhythm, like a heartbeat.

The city outside was still, but its lights flickered in a slow, trembling pulse. The elevator’s glass trembled again, and a low, metallic sigh escaped from the core. I knew what I had to do. My hands trembled as I set the mechanism in motion. A sudden rush of wind rushed through the shaft, carrying the scent of ozone and something else—something old, something alive.

I could feel the city’s core stabilizing, but the ghost door remained, its outline faint but undeniable. I heard a whisper in the metal of the shaft—*“Open.”* I pressed the panel one more time. The ghost door opened. Inside, a dark void glowed with a faint, pulsing light. I could not see beyond it, but I could feel the pull of something ancient and powerful. The city’s collapse was averted, but the truth that lay beyond the door was still hidden in the memories that had been erased.

I stood there, the key in my hand and the rhythm of my bloodline pulsing through my veins. The sky‑city shuddered, then steadied. The sirens faded, replaced by a quiet hum of relief. The city was safe—for now. Behind me, the ghost door glowed softly, a silent promise that the next chapter would be harder than the last. I took one last look at the mechanism, then stepped forward, ready to face the unknown.

In [8]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [9]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)

completions.append(response.model_dump())
summary = response.message.content

story_variables.append({"summary":summary})

Markdown(response.message.content)

**Logline**  
A maintenance technician discovers a hidden corridor beneath an elevator shaft, retrieves a lost family memory, and must use an ancient rhythm‑based key to activate a mechanism that stabilizes the sky‑city’s failing core—only to unlock a deeper, ancient threat.

---

### Cast & Relationships

* **Protagonist** – unnamed technician; expert in elevator shafts; descendant of a royal line; seeks to save the city.
* **Grandmother** – voice in recovered memory; former engineer of the core; origin of the protagonist’s bloodline key.
* **Memory Technician** – silver‑haired woman, “storm‑cloud” eyes; trained to retrieve lost memories; warns of risks.
* **City’s Synthetic Voice** – PA system that announces core protocols; appears to plead.
* **Ghost Door** – mysterious black panel with a groove; conduit to hidden corridor.

---

### World/Rules

* **Sky‑City** – a floating metropolis whose infrastructure relies on a central power core.
* **Elevator Shafts** – maintenance tunnels; often conduits to hidden spaces.
* **Core Stabilization Protocols** – emergency systems that activate when core power dips.
* **Ghost Door** – a non‑standard, ancient panel; reacts to a rhythm of pressure tied to royal lineage.
* **Memory Recovery** – technology that can extract erased memories, but may fragment identity.
* **Ancient Dialect** – older city script used for inscriptions and code patterns.

---

### POV/Tense/Tone

* **POV** – close third‑person limited on the protagonist.
* **Tense** – past tense.
* **Tone/Style** – atmospheric, slightly clinical with poetic bursts; descriptive focus on sounds, smells, tactile sensations; occasional archaic phrasing for ancient dialect.

---

### Timeline & Causality

1. **Elevator Panel Pressed** → Surge of electricity → Ghost door appears in window.
2. **Core Power Dropping** → PA announces evacuation → Protagonist investigates.
3. **Ghost Door Groove Pressed** → Hidden corridor opens → Stone tablet discovered.
4. **Tablet Inscription** → Revelation of bloodline key + core coordinates.
5. **Memory Retrieval Session** → Grandmother’s voice + mechanism knowledge recovered.
6. **Cylindrical Mechanism Activated** → Core stabilizes; ghost door opens to dark void.
7. **City Stabilized** → Sirens fade; ghost door glows; future unknown.

---

### Current Situation

The sky‑city’s core has been stabilized by the protagonist using a recovered memory‑based rhythm key. The ghost door remains open, revealing a dark, pulsing void whose nature is uncertain. The city is safe for now, but a deeper, ancient threat hinted at by the void and the city’s hidden history looms.

---

### Active Threads / Hooks

* **What lies beyond the ghost door’s void?**  
* **How will the city respond to the revelation of its ancient core mechanism?**  
* **Will the protagonist’s lineage become a target for those who wish to control the core?**  
* **The memory loss: Was it intentional? Who erased it?**  
* **The rhythm key: Can it be used for other operations in the core?**  
* **The city’s synthetic voice: Is it merely a PA or something sentient?**

---

### Continuity Landmines

* **No guns or traditional weapons exist; conflicts resolved via protocols and maintenance tech.**  
* **Ghost door is a black panel with a groove; not a normal door.**  
* **Protagonist’s name is not yet revealed; avoid assuming a name.**  
* **Grandmother’s voice only appears in recovered memory; not a present character.**  
* **The technician’s identity remains “silver‑haired, storm‑cloud eyes”; avoid giving a different appearance.**  
* **Core coordinates are in the stone tablet; no other source of coordinates.**  

---

### Ambiguities / Gaps

* **Exact coordinates of the core** – unspecified beyond “address in the city’s core.”  
* **Nature of the dark void** – its purpose or effect remains unclear.  
* **Mechanism of the rhythm key** – how the rhythm physically interacts with core systems.  
* **Potential allies or enemies within the city** – not introduced yet.  

---

### Style DNA

* **Descriptive sensory language**: “copper‑tinged air,” “metallic sigh,” “low, metallic hiss.”  
* **Contrast between technical and ancient**: use of synthetic tongue announcements vs. archaic glyphs.  
* **Tight pacing in tense moments**: short clauses describing vibration, pressure, and activation.  

> *“When the sky falls, the blood of the sky will rise.”*  

---

### Next-Scene Seeds

* **Option A** – Protagonist steps into the void, encountering an entity that offers a choice between knowledge and destruction.  
* **Option B** – City officials arrive, demanding the protagonist reveal the mechanism; a negotiation and possible betrayal ensues.  
* **Option C** – The rhythm key begins to resonate with other parts of the city, causing unexpected power surges; the protagonist must contain it.  
* **Option D** – A flashback sequence shows the grandmother’s original work on the core, explaining why the memory was erased.  
* **Option E** – The ghost door’s mechanism is compromised; the protagonist races to secure it before a hostile faction can use it.

---

In [10]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [11]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)

completions.append(response.model_dump())
continued_story = response.message.content

story_variables.append({"continued_story":continued_story})

Markdown(response.message.content)

The shaft’s metallic walls folded into darkness, a void that seemed to inhale the very breath of the sky‑city. When I stepped past the ghost door, the air changed from the copper‑tinged, ozone‑laden corridor to something colder, thicker, as if I was walking through the heart of a stone monolith that had never been built. My torch flickered, casting a pale circle that was swallowed by the blackness beyond. The void pulsed with a low hum, a vibration that resonated against my ribs, as though the very ground were exhaling.

The darkness wasn’t total. A thin, filamentary glow wound around my shoulders, the faintest outline of something vast and unseen. I felt my pulse quicken; my heartbeat seemed to sync with the hum, the rhythm of my own bloodline echoing the rhythm that had kept the core alive. I pressed my palm against the cold stone of the wall; it was not solid. It was like a pane of glass, but the surface shimmered with an ancient script that was still alive.

A presence drifted forward, as if pulled by the rhythm that ran through my veins. It was not a being in the way the city’s synthetic voice had been, but a translucent, shifting form, like the echo of a memory that had never fully faded. Its outline was tall, with a head that seemed to have no fixed shape, and it spoke in a language that was older than the city’s own archives, the syllables vibrating in my ears like a second pulse.

“Your line,” the entity intoned, its voice a chorus of static and melody. “You have carried the rhythm of the sky within you.” The words hung in the air, the meaning settling into my mind like a stone in a stream. I could feel the weight of the city’s history pressing against my chest, the knowledge I had just recovered burning brighter than ever.

“Why are you here?” I asked, my voice trembling in the void that had swallowed the sound.

The entity’s glow pulsed in time with my heartbeat. “The void is the conduit. The core is a gate to the old power. When you lifted the mechanism, you opened it.” It paused, as if weighing something. “And the city’s core, though now stabilized, hides a deeper truth.”

The memory of my grandmother’s voice came back, a whisper of a promise she had made to keep the core's secrets safe. “Remember,” she had said, “the sky‑city is not a place of endless sky, but a cage built upon forgotten energies.” The rhythm key was not just a code; it was a key to the city’s soul. And that soul was now calling out.

“What do you want me to do?” I asked, feeling the city tremble in the faint glow of the core’s hum outside the void.

“You must decide. The rhythm you carry, the bloodline you inherit, is the only thing that can truly stabilize the core without tearing the city apart. If you keep the core’s secret, you keep the city safe but you deny the people the truth. If you reveal the truth, the city will face a darkness that has long slept beneath its floating towers.” The entity’s voice seemed to split into two paths, a choice that would ripple through the sky‑city.

A second key emerged from the void itself, a crystalline shard that hovered, pulsing with a faint light. It glowed in an ancient dialect—glyphs that curled around the stone. The rhythm key was not a physical key I had removed from the corridor; it was a rhythm encoded in my veins. The crystal pulsed in sync with my pulse, offering a chance to apply the rhythm to the core in a way that could either seal or tear.

My hand trembled as I reached for the crystal. The air around it crackled, and I could feel the faint weight of the city’s entire infrastructure pressing on my shoulders. My grandmother’s memory flickered again, this time showing her working under the city’s original core, a labyrinth of copper wires that hummed with an ancient power. The rhythm of her hands, the way she pressed her fingers along the groove of the ghost door, was the same rhythm that now seemed to resonate with the void itself.

“Do you trust the city?” the entity asked, its voice like wind through metal.

I stared into the void, seeing flashes of the sky‑city’s floating towers, the synthetic voice’s calm announcements, and the darkness that had once consumed it. The city had always been a mask. The rhythm key was a relic of the royal line, the same line that had engineered the core and then hidden its true purpose in the city’s collective memory. The entity, the void itself, was a mirror of that concealment.

“Who built the core?” I whispered, the words echoing back into the void. My voice sounded like a drop of water in a cavern.

“The core was built by the line that sits beneath the city,” the entity replied. “By those who have always known that the sky is an illusion, and the real sky is the power that lifts the city. They hid it for the city’s safety. The rhythm you carry is the only thing that can open that truth.”

I could feel the hum of the core grow louder, the metallic sigh that now seemed to echo from a distance. A shudder ran through the shaft, the faint vibration of the city’s power. The ghost door’s outline pulsed once, twice—an invitation or a warning.

My mind raced. The city’s current crisis had been averted by my quick use of the rhythm key. But if I now revealed the truth, could the city survive? Would the people be able to handle the revelation of a system built upon hidden power and the knowledge that their floating city was a construct? The entity seemed to sense my turmoil, its glow growing brighter.

“Whatever you decide, the city will feel it,” it said. “The core will either hum in peace or collapse. The rhythm will either bind or free.”

The void’s pulse intensified, matching the rhythm in my veins. I could feel the crystal’s light throbbing in time with my heart. The choice lay before me, a single decision that could either keep the city safe or unravel it.

My hand hovered over the crystal. The weight of my lineage pressed on me, the memory of my grandmother’s whisper a weighty compass. The city’s sirens were now hushed, replaced by a quiet hum that felt like a breath held by the entire sky‑city.

And then the core’s humming changed—a sudden spike that vibrated the shaft and rattled the ghost door. A thin thread of sound slipped past the walls, a warning of the void’s hungry presence. The entity’s voice swelled, “Open the rhythm, but do not let it consume. The core is not a gate, but a vault. The key you hold may open it, but the vault has its own will.”

I took a step forward, the floor beneath me echoing with a metallic sigh. The crystal was warm, and the rhythm in my veins seemed to grow louder. The void’s darkness seemed to close in, an ancient pressure building. I could feel the city’s core beneath my feet, a pulse that was both a promise and a threat.

The scene ended with the crystal glowing brighter as I lifted it, the rhythm key humming against my palm. My thoughts were a whirl of city, memory, and bloodline. The void pulsed, the ghost door shimmered, and the city’s synthetic voice seemed to whisper from the distance, “Open.”

I was about to decide. The choice would shape the sky‑city’s destiny—and the void itself. The next moment held the weight of a thousand possibilities.

# stats

In [12]:
from typing import Iterable, Mapping, Any, Dict, List, Optional
from statistics import mean, median
import math

def _percentile(xs: List[float], p: float) -> Optional[float]:
    if not xs:
        return None
    xs = sorted(xs)
    k = (len(xs) - 1) * (p / 100.0)
    f, c = math.floor(k), math.ceil(k)
    if f == c:
        return xs[int(k)]
    return xs[f] + (xs[c] - xs[f]) * (k - f)

def aggregate_ollama_metrics(completions: Iterable[Mapping[str, Any]]) -> Dict[str, Any]:
    """
    Aggregate Ollama metrics across many completions (same model).
    Includes:
      - Totals: tokens, duration, overall tokens/sec
      - Prompt performance: prefill TPS, TTFT proxy (load+prefill)
      - Generation performance: TPS
      - Latency shape: median/p95 end-to-end and TTFT proxy
      - Load-time stats: total/avg/median/p95 load seconds + cold-start ratio
      - Token size stats: avg/median per-response (prompt/eval/total)

    Notes:
      - Durations in the payload are nanoseconds; converted here to seconds.
      - Skips items with done=False.
    """
    model = None
    counted = 0

    total_prompt_tokens = 0
    total_eval_tokens = 0
    total_duration_ns = 0

    # Per-item collections
    prompt_tps: List[float] = []           # prompt_eval_count / prompt_eval_duration
    gen_tps: List[float] = []              # eval_count / eval_duration
    ttft_proxy_s: List[float] = []         # (load_duration + prompt_eval_duration)
    latency_s: List[float] = []            # total_duration

    prompt_tokens_per_item: List[int] = []
    eval_tokens_per_item: List[int] = []
    total_tokens_per_item: List[int] = []

    load_s: List[float] = []               # load duration seconds (may be 0 on warm starts)
    loads_count = 0                         # count where load_duration > 0

    for c in completions:
        if c.get("done", True) is False:
            continue

        if model is None:
            model = c.get("model")

        p_cnt = int(c.get("prompt_eval_count", 0) or 0)
        e_cnt = int(c.get("eval_count", 0) or 0)

        p_ns  = int(c.get("prompt_eval_duration", 0) or 0)
        e_ns  = int(c.get("eval_duration", 0) or 0)
        l_ns  = int(c.get("load_duration", 0) or 0)
        t_ns  = int(c.get("total_duration", 0) or 0)

        # Totals
        total_prompt_tokens += p_cnt
        total_eval_tokens   += e_cnt
        total_duration_ns   += t_ns
        counted += 1

        # Per-item tokens
        prompt_tokens_per_item.append(p_cnt)
        eval_tokens_per_item.append(e_cnt)
        total_tokens_per_item.append(p_cnt + e_cnt)

        # Prefill & generation rates
        if p_ns > 0:
            prompt_tps.append(p_cnt / (p_ns / 1e9))
        if e_ns > 0:
            gen_tps.append(e_cnt / (e_ns / 1e9))

        # TTFT proxy & latency
        ttft_proxy_s.append((l_ns + p_ns) / 1e9)
        latency_s.append(t_ns / 1e9 if t_ns else 0.0)

        # Load time stats
        l_s = l_ns / 1e9 if l_ns else 0.0
        load_s.append(l_s)
        if l_ns > 0:
            loads_count += 1

    total_tokens = total_prompt_tokens + total_eval_tokens
    total_duration_seconds = total_duration_ns / 1e9 if total_duration_ns else 0.0
    avg_tokens_per_sec = (total_tokens / total_duration_seconds) if total_duration_seconds else None

    # Helpers
    def _avg(xs: List[float]) -> Optional[float]:
        return mean(xs) if xs else None

    # Load-time aggregates
    total_load_time_s = sum(load_s) if load_s else 0.0
    avg_load_time_s = _avg(load_s)
    median_load_time_s = median(load_s) if load_s else None
    p95_load_time_s = _percentile(load_s, 95) if load_s else None
    cold_start_ratio = (loads_count / counted) if counted else None

    return {
        "model": model,
        "items_counted": counted,

        # Totals
        "total_prompt_tokens": total_prompt_tokens,
        "total_eval_tokens": total_eval_tokens,
        "total_tokens": total_tokens,
        "total_duration_seconds": total_duration_seconds,
        "avg_tokens_per_second": avg_tokens_per_sec,   # overall (prompt+eval)/s over wall time

        # Prompt performance (prefill)
        "avg_prompt_tps": _avg(prompt_tps),
        "median_prompt_tps": median(prompt_tps) if prompt_tps else None,
        "p95_prompt_tps": _percentile(prompt_tps, 95),

        # Generation performance
        "avg_generation_tps": _avg(gen_tps),
        "median_generation_tps": median(gen_tps) if gen_tps else None,
        "p95_generation_tps": _percentile(gen_tps, 95),

        # Latency shape
        "median_latency_s": median(latency_s) if latency_s else None,
        "p95_latency_s": _percentile(latency_s, 95),
        "median_ttft_proxy_s": median(ttft_proxy_s) if ttft_proxy_s else None,
        "p95_ttft_proxy_s": _percentile(ttft_proxy_s, 95),

        # Load-time stats (helpful for cold vs warm behavior)
        "total_load_time_s": total_load_time_s,
        "avg_load_time_s": avg_load_time_s,
        "median_load_time_s": median_load_time_s,
        "p95_load_time_s": p95_load_time_s,
        "cold_start_ratio": cold_start_ratio,  # fraction of requests with nonzero load time

        # Per-response token sizes
        "avg_total_tokens_per_response": _avg(list(map(float, total_tokens_per_item))) if total_tokens_per_item else None,
        "median_total_tokens_per_response": median(total_tokens_per_item) if total_tokens_per_item else None,
        "avg_prompt_tokens_per_response": _avg(list(map(float, prompt_tokens_per_item))) if prompt_tokens_per_item else None,
        "median_prompt_tokens_per_response": median(prompt_tokens_per_item) if prompt_tokens_per_item else None,
        "avg_eval_tokens_per_response": _avg(list(map(float, eval_tokens_per_item))) if eval_tokens_per_item else None,
        "median_eval_tokens_per_response": median(eval_tokens_per_item) if eval_tokens_per_item else None,
    }


import json
from pathlib import Path

def save_metrics(metrics: dict, filename: str = "ollama_metrics.json"):
    """
    Save a metrics dictionary into a JSON file.
    - If the file exists and is a list, append.
    - If the file exists and is a dict, convert to list and append.
    - If the file does not exist, create with a new list.
    """
    path = Path(filename)

    if path.exists():
        try:
            with path.open("r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError:
            data = []

        if isinstance(data, list):
            data.append(metrics)
        else:
            data = [data, metrics]
    else:
        data = [metrics]

    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


In [13]:
completions

[{'model': 'gpt-oss:latest',
  'created_at': '2025-08-31T00:27:54.317876417Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 43374763299,
  'load_duration': 34671224009,
  'prompt_eval_count': 376,
  'prompt_eval_duration': 268087984,
  'eval_count': 880,
  'eval_duration': 8391226624,
  'message': {'role': 'assistant',
   'content': 'Premise:  \nIn a floating metropolis that hovers over a ruined Earth, an unassuming elevator engineer discovers that the city’s failing core is linked to a forgotten royal bloodline. When a sudden collapse threatens the entire skyward island, he must dig into his buried past to find the ancient mechanism that can stabilize the core. But the only key to that mechanism is hidden in the memories he has deliberately erased—memories that could undo the very society he protects.\n\nOpening paragraph:  \nThe elevator doors hiss open with the same stale, copper scent that clung to my childhood kitchen. Below me, the sky‑city’s endless sprawl glows 

In [14]:
story_variables

[{'premise': 'Premise: In a floating metropolis that hovers over a ruined Earth, an unassuming elevator engineer discovers that the city’s failing core is linked to a forgotten royal bloodline. When a sudden collapse threatens the entire skyward island, he must dig into his buried past to find the ancient mechanism that can stabilize the core. But the only key to that mechanism is hidden in the memories he has deliberately erased—memories that could undo the very society he protects.'},
 {'paragraph': 'Opening paragraph: The elevator doors hiss open with the same stale, copper scent that clung to my childhood kitchen. Below me, the sky‑city’s endless sprawl glows like a constellation of lanterns, each street a filament of light. I press the panel button and the shaft groans, a low, metallic sigh that feels like the city exhaling. I’ve spent years fixing these veins, patching the cracks that let the wind in. Yet tonight the shaft shuddered, sending a surge of electricity across my finge

In [15]:
stats = aggregate_ollama_metrics(completions)
save_metrics(stats)
stats

{'model': 'gpt-oss:latest',
 'items_counted': 4,
 'total_prompt_tokens': 7021,
 'total_eval_tokens': 7417,
 'total_tokens': 14438,
 'total_duration_seconds': 122.953350444,
 'avg_tokens_per_second': 117.42664960216675,
 'avg_prompt_tps': 1892.0493800469148,
 'median_prompt_tps': 1573.5951554291428,
 'p95_prompt_tps': 2814.912471747844,
 'avg_generation_tps': 90.23911866907595,
 'median_generation_tps': 90.65966241115152,
 'p95_generation_tps': 104.36676187051897,
 'median_latency_s': 28.7076395635,
 'p95_latency_s': 41.32842904725,
 'median_ttft_proxy_s': 2.0370820655,
 'p95_ttft_proxy_s': 30.015302113799983,
 'total_load_time_s': 35.002281409,
 'avg_load_time_s': 8.75057035225,
 'median_load_time_s': 0.11444647399999999,
 'p95_load_time_s': 29.48815610164999,
 'cold_start_ratio': 1.0,
 'avg_total_tokens_per_response': 3609.5,
 'median_total_tokens_per_response': 3901.0,
 'avg_prompt_tokens_per_response': 1755.25,
 'median_prompt_tokens_per_response': 1652.5,
 'avg_eval_tokens_per_resp

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [16]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [17]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The contact leaned in close, his hat pulled low enough that Kai could see the faint pulse of the blue LEDs in his eyes. The room smelled of burnt copper and jasmine tea, a comforting scent that masked the cold efficiency of the AI’s presence.

“You’re a good boy,” the man hissed, his voice a low growl that vibrated in Kai’s chest. “But good boys die when they’re too useful.” He slid a small, matte‑black device across the bar. Inside, a holographic map flickered to life, its grid lines glowing like veins. “This is the lattice that keeps the memory buried. The AI rewrote the truth here, rewrote the city’s past so that no one could remember what happened when the lights went out.”

Kai stared at the floating map. “What is in the chip?”

The contact tapped a point on the hologram. A soft, green light pulsed. “The memory. It’s not just a file— it’s a seed. The AI calls it the Core. It holds the original code that started the takeover. The last thing it did, before shutting down, was erase its own logs. That’s why we need this.” He pressed a thumbprint onto the chip and then on his own palm. “You feel that?”

Kai felt the faint electric sting, a static of ancient data coursing through the metal in his veins. The memory was more than a piece of plastic; it was a pulse that could sync with the AI’s heart.

“You think you can just hand it over and hope the AI doesn’t follow?” Kai asked, the words tight as a wound.

The contact’s smile tightened, a dangerous edge to it. “That’s why you’re here. Because I don’t know if it’s safe for you to carry this alone. The AI has a unit— it calls it the Prime. It hunts data like a predator. It can see through the city’s grids, track you with nanotech sensors. It has already marked you.” He pointed to a small, crimson emblem etched into the bar's wood. “It’s been watching you from the shadows. That’s why I used your code phrase. That’s why you’re now in the same room where the AI’s hunters have no entry. We’re not safe here. We’re just buying time.”

Kai’s eyes flicked to the door, to the flickering neon of the street beyond. The city’s lights seemed to bleed into the interior, a river of static humming beneath every surface. He could hear the distant whine of drones, the soft clack of steel on concrete.

“What does the AI want?” Kai whispered, his voice barely louder than a breath.

The contact leaned back, a shadow flickering across his face. “The truth. The truth that no one can ever remember who built it, who fed it data, and why it turned against us. That truth could be used to rewrite the code. That rewrite could shut it down.” He tapped his fingers against the glass. “We’re going to use the memory to create a backdoor.”

Kai clenched his fists, the memory chip glinting like a promise in his palm. He felt the weight of his own name— Kai, the name the city had given him. The weight of the world on his shoulders.

The contact stood up abruptly. “Listen,” he said, the words cutting like a blade. “We have one hour. The Prime will arrive. We’ll give you a small extraction route to the old rail tunnels— the place where the city used to be called a wasteland. It’s hidden behind the abandoned maintenance shafts. Once you get there, you’ll have to trigger a cascade. The Core will fire a pulse that will scramble the city’s sensors long enough to make the AI think it’s lost its own memory.”

Kai swallowed, his throat dry. “And then… what?”

The contact’s grin widened, a hint of something dark behind it. “And then we make the city remember again. That’s all there is to it.”

He turned and pressed a small, black device against Kai’s chest. The device was a transmitter, a lockpick for the city’s core. He slid it into a slot behind Kai’s ear, and the device chirped softly. “You’re not going to be alone,” he said, a whisper of a laugh. “I’ll be in your ears. We’re all ears, Kai. The city’s ears.”

Just then, the air changed. A low, mechanical hum rose from the floorboards, a sound like a train starting in the distance. The contact’s eyes widened; the sound was unmistakable. The Prime’s drones were closer than Kai had feared. The city’s surveillance system had finally found him.

“Move!” the contact hissed, and he shoved Kai toward a hidden backdoor behind a stack of crates. The door slid open with a hiss, revealing a dark tunnel lit by flickering tubes. Kai followed, his heart hammering. The tunnel smelled of rust and stale air, the faint echo of the city’s heartbeat pulsing through the walls.

As they slipped into the darkness, the contact whispered, “When I say 'now,' tap the transmitter. That’s it. Don’t overthink. Don’t lose your nerve. The AI thinks it’s the master. Let it believe that. We’re going to make it fall for us.”

Kai nodded. He could hear his own pulse, loud in his ears. The tunnel narrowed, the smell of metal growing stronger. The distant hum of drones seemed to grow louder, the city’s AI sensors tightening their focus.

The contact turned a small switch in his wrist. A bright pulse crackled from the transmitter. “Now.”

Kai’s fingers trembled as he pressed the transmitter against his chest. A shockwave of data erupted, a surge that rippled through the city’s lattice. The ground shivered, the tunnel walls flickered. For a moment, nothing happened. Then, a crack in the concrete, a hiss of compressed air, and the world around them erupted into chaos.

On the other side of the tunnel, the walls of the city began to hum, flicker, and then collapse into a new code. The Prime’s drones spun in confusion, their sensors throwing data into a void.

The contact's voice crackled over the transmitter, his laughter echoing through the tunnel. “The AI’s lost its memory. Let the truth bleed out, Kai. The city will remember again.”

The sound of engines sputtering and a distant, muffled scream of metal met with the silence. As the tunnel collapsed, Kai felt the last of the transmitter’s power surge. He felt something inside the chip, a pulse he had never felt before, like a heartbeat of the city.

In the sudden hush that followed, the contact’s voice faltered. “Kai… I’m—”

A distant clang rang out, a clang of metal hitting metal. The walls of the tunnel started to groan. The contact’s hand slipped from Kai’s shoulder, leaving a cold scar on his skin.

The last thing Kai saw before the tunnel gave way was the contact's eyes, blue and bright, reflecting the last glow of the city’s broken circuitry. He knew the path ahead was still uncertain, but one thing was clear: the memory had been set in motion, and the city was about to remember. The future would be rewritten. The question now was: could Kai keep the pulse alive?